In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import matplotlib.animation as animation
import glob, re, os
import subprocess

from smt.applications.mixed_integer import (
    MixedIntegerContext,
    FLOAT,
    ENUM,
    ORD,
)

import scipy.linalg as lng

from smt.applications import EGO
from smt.surrogate_models import KRG
from smt.sampling_methods import LHS


import scienceplots
plt.style.use('default')
plt.style.use(['science', 'high-vis', 'grid'])

In [5]:
PATH_COST_CPP = "/work/utox/users/helleboid_work_utox/OptiDopi/build/apps/spad_cost_function"
RES_FILE = "RES/res_cost_func.csv"

os.makedirs("RES/", exist_ok=True)

def function_test_1d(length_intrinsic):
    doping_acceptor = 17.0
    subprocess.run([PATH_COST_CPP, f"{length_intrinsic}", f"{doping_acceptor}", RES_FILE])
    length_intrinsic,doping_acceptor, BV,BrP,DW,BV_cost,BP_cost,DW_cost,total_cost = np.loadtxt(RES_FILE, delimiter=",", unpack=True)
    print(f)
    return total_cost

def function_test_2d(PARAMS):
    #print(PARAMS)
    Y = []
    for idx in range(len(PARAMS)):
        length_intrinsic = PARAMS[idx,0]
        #print(f'{length_intrinsic=}')
        log_doping_acceptor =  PARAMS[idx,1]
        subprocess.run([PATH_COST_CPP, f"{length_intrinsic}", f"{log_doping_acceptor}", RES_FILE])
        length_intrinsic,doping_acceptor, BV,BrP,DW,BV_cost,BP_cost,DW_cost,total_cost = np.loadtxt(RES_FILE, delimiter=",", unpack=True)
        Y.append(total_cost)
        print(f"Total cost: {total_cost}")
    return np.array(Y)

vfunc = np.vectorize(function_test_1d)

vfunc2d = np.vectorize(function_test_2d)


In [6]:
min_length_intrinsic = 0.0
max_length_intrinsic = 1.0
min_log_acceptor = 16.5
max_log_acceptor = 19.0


n_iter = 1000
xtypes = [FLOAT, FLOAT]
#mixint = MixedIntegerContext(xtypes, xlimits)
xlimits = np.array([[min_length_intrinsic, max_length_intrinsic], [min_log_acceptor, max_log_acceptor]])

qEI = "KB"
sm = KRG(print_global=False)
mixint = MixedIntegerContext(xtypes, xlimits)
n_doe = 5
sampling = LHS(xlimits=xlimits)
xdoe = sampling(n_doe)
ydoe = function_test_2d(xdoe)



Total cost: 2055.365
Total cost: 852.2339
Total cost: 10000000000.0
Total cost: -45.5323
Total cost: 1518.021


In [7]:
criterion = "SBO"  #'EI' or 'SBO' or 'LCB'
ego = EGO(n_iter=n_iter, criterion=criterion, xdoe=xdoe, xlimits=xlimits, xtypes=xtypes, qEI=qEI,
          enable_tunneling=True)
x_opt, y_opt, _, x_data, y_data = ego.optimize(fun=function_test_2d)

Total cost: 2055.365
Total cost: 852.2339
Total cost: 10000000000.0
Total cost: -45.5323
Total cost: 1518.021
Total cost: 752.4807
Total cost: 310.5109
Total cost: 2344.167
Total cost: 510.6477
Total cost: 1152.474
Total cost: 1412.58
Total cost: 1531.083
Total cost: 1525.807
Total cost: 1191.893
Total cost: 2066.726
Total cost: 2270.89
Total cost: 394.4212
Total cost: 2812.146
Total cost: 2004.691
Total cost: 40.35439
Total cost: 3976.141
Total cost: 347.0906
Total cost: -30.90319
Total cost: 3702.385
Total cost: 78.79398
Total cost: 637.0199
Total cost: 896.4461
Total cost: 903.5274
Total cost: 21.16015
Total cost: 397.4981
Total cost: 2471.053
Total cost: 60.266
Total cost: 188.3946
Total cost: 188.3946
Total cost: 172.7131
Total cost: 1327.968
Total cost: 31.55685
Total cost: 1527.364
Total cost: 1527.364
Total cost: 787.6776
Total cost: 1511.097
Total cost: 2875.247
Total cost: -39.1144
Total cost: 1102.752
Total cost: 790.3874
Total cost: 1906.942
Total cost: -59.62506
Total cost

/home/hellebor/my_utox/anaconda3/lib/python3.8/site-packages/smt/surrogate_models/krg_based.py:211: UserWarning: Warning: multiple x input features have the same value (at least same row twice).
  warnings.warn("Warning: multiple x input features have the same value (at least same row twice).")


Total cost: 169.1899


/home/hellebor/my_utox/anaconda3/lib/python3.8/site-packages/smt/surrogate_models/krg_based.py:211: UserWarning: Warning: multiple x input features have the same value (at least same row twice).
  warnings.warn("Warning: multiple x input features have the same value (at least same row twice).")


Total cost: 737.5495


...

Total cost: 536.4288


/home/hellebor/my_utox/anaconda3/lib/python3.8/site-packages/smt/surrogate_models/krg_based.py:211: UserWarning: Warning: multiple x input features have the same value (at least same row twice).
  warnings.warn("Warning: multiple x input features have the same value (at least same row twice).")


Total cost: 536.4288


/home/hellebor/my_utox/anaconda3/lib/python3.8/site-packages/smt/surrogate_models/krg_based.py:211: UserWarning: Warning: multiple x input features have the same value (at least same row twice).
  warnings.warn("Warning: multiple x input features have the same value (at least same row twice).")


Total cost: -56.80642


In [ ]:
print("Minimum in x={} with f(x)={:.1f}".format(x_opt, float(y_opt)))
print("Minimum in typed x={}".format(ego.mixint.cast_to_mixed_integer(x_opt)))

min_ref = -15
mini = np.zeros(n_iter)
for k in range(n_iter):
    mini[k] = np.log(np.abs(np.min(y_data[0 : k + n_doe - 1]) - min_ref))
x_plot = np.linspace(1, n_iter + 0.5, n_iter)
u = max(np.floor(max(mini)) + 1, -100)
l = max(np.floor(min(mini)) - 0.2, -10)
fig = plt.figure()
axes = fig.add_axes([0.1, 0.1, 0.8, 0.8])
axes.plot(x_plot, mini, color="r")
axes.set_ylim([l, u])
plt.title("minimum convergence plot", loc="center")
plt.xlabel("number of iterations")
plt.ylabel("log of the difference w.r.t the best")
plt.show()